In [280]:
import torch
import copy

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import numpy as np
import glob, os
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from torch import nn, optim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
import torch.nn.functional as F

In [244]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dir = './models/*'
model_files = glob.glob(dir)
device

device(type='cuda')

In [245]:
class Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(Encoder, self).__init__()
        
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
        
        self.rnn1 = nn.LSTM(
          input_size=n_features,
          hidden_size=self.hidden_dim,
          num_layers=1,
          batch_first=True
        )
        self.rnn2 = nn.LSTM(
          input_size=self.hidden_dim,
          hidden_size=embedding_dim,
          num_layers=1,
          batch_first=True
        )
    def forward(self, x):

        x = x.reshape((1, self.seq_len, self.n_features))
        
        x, (_, _) = self.rnn1(x)
        x, (hidden_n, _) = self.rnn2(x)
        
        return hidden_n.reshape((self.seq_len, self.embedding_dim))

In [246]:
class Decoder(nn.Module):
    def __init__(self, seq_len, input_dim=64, n_features=1):
        super(Decoder, self).__init__()
        
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
       
        self.rnn1 = nn.LSTM(
            input_size=input_dim,
            hidden_size=input_dim,
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=input_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
        )
        
        self.output_layer = nn.Linear(self.hidden_dim, n_features)
    def forward(self, x):
        x = x.repeat(self.seq_len, self.n_features)
        x = x.reshape((self.n_features, self.seq_len, self.input_dim))
        
        x, (hidden_n, cell_n) = self.rnn1(x)
        x, (hidden_n, cell_n) = self.rnn2(x)
        x = x.reshape((self.n_features, self.hidden_dim))
        return self.output_layer(x)

In [247]:
class RecurrentAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(RecurrentAutoencoder, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [248]:
model_files[0]

'./models/model_onehot_x1data_L1_epoch200.pth'

In [249]:
model_onehot_L1 = torch.load(model_files[0])

In [250]:
alpha = list(string.ascii_uppercase)
chr2index = {alpha[i]:i for i in range(len(alpha))}

In [251]:
def chr2OH(alphabet):
    oh = [0 for i in range(len(alpha))]
    index = chr2index[alphabet]
    oh[index] = 1
    return oh

In [252]:
dir = './datasets/fsm/0graph*'
# file read
all_names = []
all_data = []
sequence_length = []
alpha = list(string.ascii_uppercase)
data_length = len(glob.glob(dir))

files = glob.glob(dir)
files

['./datasets/fsm/0graph1.txt',
 './datasets/fsm/0graph0.txt',
 './datasets/fsm/0graph2.txt']

In [253]:
files = glob.glob(dir)
for file in files:
    datasets = []
    for rf in open(file, 'r'):
        (u, v, w) = rf[1:-2].split(', ')
        datasets.append(chr2OH(u[1]) + chr2OH(v[1]) +[float(w)])
        #datasets.append([chr2index[u[1]], chr2index[v[1]], float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)


In [254]:
max_sequence_length = 5
#zeros = np.zeros(53)
zeros = [0 for i in range(53)]
for ind, data in enumerate(all_data):
    if len(data) != max_sequence_length:
        for i in range(len(data), max_sequence_length):
            all_data[ind].append(zeros)
       

all_data = np.array([np.array(arr) for arr in all_data])
                       

In [255]:
all_d = []
for ind, data in enumerate(all_data):
    all_d.append(data.flatten())
all_d
len(all_d)

3

In [256]:

def create_dataset_1(nparrays):
    dataset = [torch.tensor(s).unsqueeze(0).float() for s in nparrays]
  #step_per_epoch, seq_len, llen, n_features = torch.stack(dataset).shape
    #n_seq, m ,seq_len, n_features = print(torch.stack(dataset).shape)
    print(torch.stack(dataset).shape)
    n_seq, seq_len, n_features = torch.stack(dataset).shape
    #print( n_seq, m , seq_len, n_features )
    return dataset, seq_len, n_features

In [260]:
ts, seq_len_1d , n_features_1d  = create_dataset_1(all_d)
ss  = [torch.tensor(s).unsqueeze(0).float() for s in all_d]
T = ts[1]
F = ts[0]
M = ts[2]

torch.Size([3, 1, 265])


In [261]:
M

tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000, 30.9065,  0.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [239]:
def predict(model, dataset):
    predictions, losses = [], []
    criterion = nn.L1Loss(reduction='mean').to(device)
    with torch.no_grad():
        model = model.eval()
        for seq_true in dataset:
            seq_true = seq_true.to(device)
            seq_pred = model(seq_true)
            print(seq_pred.shape)
            loss = criterion(seq_pred, seq_true)
            print(loss)
            print(seq_pred.cpu().numpy().flatten())
            predictions.append(seq_pred.cpu().numpy().flatten())
            losses.append(loss.item())
    return predictions, losses

In [275]:
prT, _ = predict(model_onehot_L1, T)

torch.Size([265, 265])
tensor(0.0753, device='cuda:0')
[ 9.0990865e-01  1.3487136e-02  5.1256940e-03 ... -7.9503423e-04
 -2.3353663e-03  2.4692950e+00]


In [276]:
prF, _ = predict(model_onehot_L1, F)

torch.Size([265, 265])
tensor(0.6906, device='cuda:0')
[ 0.70929873  0.12725899  0.02833471 ... -0.01676553 -0.03422892
  7.3599625 ]


In [278]:
prM,_= predict(model_onehot_L1, M)
prM

torch.Size([265, 265])
tensor(0.1356, device='cuda:0')
[ 9.3335974e-01  1.0611330e-01 -1.5971115e-01 ... -1.8545599e-03
 -1.6880536e-02  4.2876825e+00]


[array([ 9.3335974e-01,  1.0611330e-01, -1.5971115e-01, ...,
        -1.8545599e-03, -1.6880536e-02,  4.2876825e+00], dtype=float32)]

In [284]:
euclidean_distances(prT, prF)

array([[3117.7268]], dtype=float32)

In [290]:
euclidean_distances(prT, prM)

array([[208.76878]], dtype=float32)

In [287]:
euclidean_distances(prF, prM)

array([[2959.233]], dtype=float32)

In [282]:
cosine_similarity(prT, prF)

array([[0.8656398]], dtype=float32)

In [286]:
cosine_similarity(prT, prM)

array([[0.94269073]], dtype=float32)

In [291]:
cosine_similarity(prF, prM)

array([[0.8808979]], dtype=float32)